### 获取数据

In [3]:
import requests
res = requests.get('https://book.douban.com/top250/')
print(res) #碰见状态码 418，说明我们模仿得欠些火候。

<Response [418]>


[MDN网页开发手册](https://developer.mozilla.org/zh-CN/docs/Web/HTTP/Status/418)

headers 中文名叫 消息头。无论是浏览器向服务器发送请求，还是服务器向浏览器作出响应，这中间都会通过 headers 传递附加信息。这些附加信息中就包含了我们爬虫程序伪装浏览器的关键。

In [4]:
import requests

# 定制消息头
headers = {
  # 消息头中有一项附加信息 user-agent
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}
# 携带消息头向服务器发送请求
res = requests.get('https://book.douban.com/top250/', headers=headers)
print(res)

<Response [200]>


### 获取相应内容

requests 库早已把响应内容转换为 字符串 类型，保存到 Response 对象的 text 属性中，我们能很轻松地通过 . 运算符访问到：

In [5]:
print(res.text)



<!DOCTYPE html>
<html lang="zh-cmn-Hans" class="ua-mac ua-webkit book-new-nav">
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
  <title>豆瓣读书 Top 250</title>
  
<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\s+|\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call(e,o)}catch(e){t

### 解析数据

bs4 中的 4 是版本号，bs 是它名称缩写，全称叫 BeautifulSoup——取自刘易斯·卡罗尔在《爱丽丝梦游仙境》里的同名诗歌。正如诗歌所言，BeautifulSoup 尝试化平淡为神奇，通过将网页源代码解析成 BeautifulSoup 对象，把结构复杂的网页代码内部组织关系捋得清清楚楚。

创建 BeautifulSoup 对象 时，它会把 HTML/XML 文档解析成 树形结构
BeautifulSoup 类继承自 Tag 类，因此 BeautifulSoup 类和 Tag 类有许多共用的方法。

创建 BeautifulSoup 对象时需要传入两个参数：
所需解析的 HTML 代码，即响应的文本内容（res.text）；
用于解析 HTML 代码的 解析器，课程内使用的是 Python 内置解析器 html.parser（parser：解析器）。

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
print(type(soup))
# 打印 soup 变量值
print(soup)

<class 'bs4.BeautifulSoup'>

<!DOCTYPE html>

<html class="ua-mac ua-webkit book-new-nav" lang="zh-cmn-Hans">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>豆瓣读书 Top 250</title>
<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\s+|\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{

BeautifulSoup 对象 内部结构像一棵倒着生长的树，树根在上面，枝叶在下面，每个节点都对应着 HTML 代码中的一个元素。顺着这棵 HTML 文档树，我们就能定位到某个或某群具有相同特征的元素，从而提取出元素文本内容——也就是我们所需的数据。

我们解析得到的 BeautifulSoup 对象，本质上就是这样一棵 HTML 文档树。树中每个节点都被抽象为了 Tag 对象。还记得吗？BeautifulSoup 类是继承自 Tag 类的，所以 BeautifulSoup 对象不仅代表着整棵 HTML 文档树，还代表着根节点。

### CSS选择器

- div 元素 是 HTML 中一种常见的元素。它没有语义，不指代特定类型内容。不过它是一个绝佳的“容器”，能将 HTML 代码划分为不同的区域，方便使用 class 属性、id 属性定义内容格式。

元素与元素之间的关系可以分为 祖先 – 后代、祖先 – 直接后代、兄弟 – 兄弟 三类，因此两个元素之间的组合关系也分为这三种：
- A B：检索 A 元素 后代 中的所有 B 类型元素；
- A > B：检索 A 元素 直接后代 中所有 B 类型元素；
- A ~ B：检索 A 元素 兄弟 中所有 B 类型元素。

注意啦，这两句诗虽然都是 p 元素，彼此互为兄弟，但它们之间也是有长幼之分的。紧挨着父元素的 <p id="first">床前明月光，疑是地上霜。</p> 是长兄，后面的 <p id="second">举头望明月，低头思故乡。</p> 是幼弟。选择器 p ~ p 的实际含义是：在长兄 p 元素的所有兄弟中，寻找类型为 p 的元素，因此只会选中幼弟，而不会选中长兄。

- .class_name：表示检索所有 class 属性值为 class_name 的元素；
- #id_name：表示检索所有 id 属性值为 id_name 的元素。

其中靠 class 属性值检索的选择器被称为 类选择器，靠 id 属性值检索的被称为 ID 选择器。它们与元素选择器一起并称为 基本选择器。

如果一个元素有多个 class 属性值，或者既有 class 属性又有 id 属性，比如 <p class="c1 c2" id="d1"></p>，那么在检索它时，我们可以将多个基本选择器连接起来，写成 p.c1.c2#d1。

注意啦，在连接多个基本选择器时，基本选择器与基本选择器之间是 没有空格的！假如你在中间加了空格，CSS 会以为你在描述 后代 关系哦。在实战过程中一定要多检查几遍哦

基本选择器：通过元素的 类型、class 属性值、id 属性值 检索；
组合器：通过元素和元素之间的 关系 检索，分为 直接后代组合器、后代组合器 和 兄弟组合器。

### select() 方法

In [9]:
book_name_tags = soup.select('div.pl2 a')
print(book_name_tags)
print(type(book_name_tags[0]))
# 输出：<class 'bs4.element.Tag'>

[<a href="https://book.douban.com/subject/1007305/" onclick="&quot;moreurl(this,{i:'0'})&quot;" title="红楼梦">
                红楼梦

                
              </a>, <a href="https://book.douban.com/subject/4913064/" onclick="&quot;moreurl(this,{i:'1'})&quot;" title="活着">
                活着

                
              </a>, <a href="https://book.douban.com/subject/4820710/" onclick="&quot;moreurl(this,{i:'2'})&quot;" title="1984">
                1984

                
              </a>, <a href="https://book.douban.com/subject/6082808/" onclick="&quot;moreurl(this,{i:'3'})&quot;" title="百年孤独">
                百年孤独

                
              </a>, <a href="https://book.douban.com/subject/6518605/" onclick="&quot;moreurl(this,{i:'4'})&quot;" title="三体全集">
                三体全集

                
                  <span style="font-size:12px;"> : 地球往事三部曲 </span>
</a>, <a href="https://book.douban.com/subject/1068920/" onclick="&quot;moreurl(this,{i:'5'})&quot;" title="飘">
      

Tag对象

是的，Tag 类也有 select() 方法——这是因为，BeautifulSoup 类中的 select() 方法就是继承自 Tag 类的。除此之外，我们还能通过 Tag 对象的 text 属性访问到该元素的 元素内容，通过 Tag对象['元素属性名称'] 形式访问到该元素的某个属性的值。

In [10]:
book = book_name_tags[0]
print(book)

<a href="https://book.douban.com/subject/1007305/" onclick="&quot;moreurl(this,{i:'0'})&quot;" title="红楼梦">
                红楼梦

                
              </a>


In [15]:
print(book.text)
print(book['title'])
print(book['href'])


                红楼梦

                
              
红楼梦
https://book.douban.com/subject/1007305/


In [16]:
book_info_tags = soup.select('p.pl')
print(book_info_tags)

[<p class="pl">[清] 曹雪芹 著 / 人民文学出版社 / 1996-12 / 59.70元</p>, <p class="pl">余华 / 作家出版社 / 2012-8-1 / 20.00元</p>, <p class="pl">[英] 乔治·奥威尔 / 刘绍铭 / 北京十月文艺出版社 / 2010-4-1 / 28.00</p>, <p class="pl">[哥伦比亚] 加西亚·马尔克斯 / 范晔 / 南海出版公司 / 2011-6 / 39.50元</p>, <p class="pl">刘慈欣 / 重庆出版社 / 2012-1-1 / 168.00元</p>, <p class="pl">[美国] 玛格丽特·米切尔 / 李美华 / 译林出版社 / 2000-9 / 40.00元</p>, <p class="pl">J.K.罗琳 (J.K.Rowling) / 苏农 / 人民文学出版社 / 2008-12-1 / 498.00元</p>, <p class="pl">[明] 罗贯中 / 人民文学出版社 / 1998-05 / 39.50元</p>, <p class="pl">林奕含 / 北京联合出版公司 / 2018-2 / 45.00元</p>, <p class="pl">[英] 乔治·奥威尔 / 荣如德 / 上海译文出版社 / 2007-3 / 10.00元</p>, <p class="pl">[英] 阿·柯南道尔 / 丁钟华 等 / 群众出版社 / 1981-8 / 53.00元/68.00元</p>, <p class="pl">[日] 东野圭吾 / 刘姿君 / 南海出版公司 / 2013-1-1 / 39.50元</p>, <p class="pl">[法] 圣埃克苏佩里 / 马振聘 / 人民文学出版社 / 2003-8 / 22.00元</p>, <p class="pl">金庸 / 生活·读书·新知三联书店 / 1994-5 / 96.00元</p>, <p class="pl">（丹麦）安徒生 / 叶君健 / 人民文学出版社 / 1997-08 / 25.00元</p>, <p class="pl">三毛 / 哈尔滨出版社 / 2003-8 / 15.80元</p>, <p class="pl">[美] 哈珀·李 / 高红

In [23]:
info = book_info_tags[0]
info.text.split('/')

['[清] 曹雪芹 著 ', ' 人民文学出版社 ', ' 1996-12 ', ' 59.70元']

In [17]:
# 遍历所有书籍信息元素
for info_tag in book_info_tags:
  # 获取书籍信息
  info = info_tag.text
  # 按“ / ”分割字符串
  info_list = info.split(' / ')
  # 结果列表中第一项为作者信息
  author = info_list[0]
  # 倒数第三项为出版社信息
  publisher = info_list[-3]
  print(author, publisher)

[清] 曹雪芹 著 人民文学出版社
余华 作家出版社
[英] 乔治·奥威尔 北京十月文艺出版社
[哥伦比亚] 加西亚·马尔克斯 南海出版公司
刘慈欣 重庆出版社
[美国] 玛格丽特·米切尔 译林出版社
J.K.罗琳 (J.K.Rowling) 人民文学出版社
[明] 罗贯中 人民文学出版社
林奕含 北京联合出版公司
[英] 乔治·奥威尔 上海译文出版社
[英] 阿·柯南道尔 群众出版社
[日] 东野圭吾 南海出版公司
[法] 圣埃克苏佩里 人民文学出版社
金庸 生活·读书·新知三联书店
（丹麦）安徒生 人民文学出版社
三毛 哈尔滨出版社
[美] 哈珀·李 译林出版社
鲁迅 人民文学出版社
当年明月 中国海关出版社
王小波 中国青年出版社
[意] 埃莱娜·费兰特 人民文学出版社
[意] 埃莱娜·费兰特 人民文学出版社
钱穆 生活·读书·新知三联书店
[法] 阿尔贝·加缪 上海译文出版社
[以色列] 尤瓦尔·赫拉利 中信出版社
